In [90]:
# import dependencies 
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pymongo
import pandas as pd
import time
import shutil
from selenium import webdriver

In [91]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [92]:
#create path, use acutal path as shown above. 
executbale_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executbale_path, headless=False)

In [93]:
url = 'https://mars.nasa.gov/news/?'
browser.visit(url)
time.sleep(2)

In [94]:
#scrape through with soup :) 
html = browser.html
soup = bs(html, "html.parser")
news_title = soup.find('div',class_ = 'content_title').text.strip()
news_p = soup.find('div', class_="article_teaser_body").text.strip()
print(f'The Current News Title: {news_title}.')
print(f'The opening to the paragraph is: {news_p}')
      


The Current News Title: Mars InSight Lander's 'Mole' Pauses Digging.
The opening to the paragraph is: After the mission's heat probe began hammering last week, it appears to have hit a rock. The team is analyzing data before they hammer again.


# JPL Mars Space Images - Featured Image

In [137]:
#scrape through JPL to pull latest photos :) gotta stick to master splinter yo.
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [138]:
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(4) 
browser.click_link_by_partial_text('more info')


In [142]:
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, "html.parser")
img_path = jpl_soup.find('figure', class_='lede').a['href']
featured_image_url = "https://www.jpl.nasa.gov" + img_path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18826_hires.jpg


# Mars Weather from Twitter

In [101]:
#straight up shout out to medium for comeing through with this. 

twt_url = u'https://twitter.com/marswxreport?lang=en'
re = requests.get(twt_url)
twt_soup = bs(re.text, 'html.parser')

mars_weather= [p.text.strip() for p in twt_soup.find_all('p',class_='TweetTextSize')]
mars_weather

['InSight sol 99 (2019-03-07) low -95.8ºC (-140.4ºF) high -13.3ºC (8.1ºF)\nwinds from the SW at 4.2 m/s (9.5 mph) gusting to 12.0 m/s (26.8 mph)\npressure at 7.20 hPapic.twitter.com/XRi0faFSv5',
 'InSight sol 98 (2019-03-06) low -94.6ºC (-138.4ºF) high -14.8ºC (5.3ºF)\nwinds from the SW at 4.1 m/s (9.2 mph) gusting to 10.8 m/s (24.2 mph)\npressure at 7.20 hPa',
 "A few days ago @BadAstronomer made a post about the median Earth.  I thought:  that's cool; someone should do that for Mars.  But I realized, damn, the person that would do that is me.  So here's the median #Mars from 4 different years.https://twitter.com/BadAstronomer/status/1102960024110882816\xa0…",
 'InSight sol 97 (2019-03-05) low -95.3ºC (-139.6ºF) high -14.8ºC (5.3ºF)\nwinds from the SW at 4.9 m/s (10.9 mph) gusting to 12.9 m/s (28.9 mph)\npressure at 7.20 hPapic.twitter.com/Q9QivIo4ls',
 'InSight sol 96 (2019-03-05) low -95.9ºC (-140.6ºF) high -19.1ºC (-2.4ºF)\npressure at 7.20 hPapic.twitter.com/SFpZrpPEZI',
 'InSight

# Mars Facts

In [102]:
# Panda scraping.... nuff said
#o scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_url = "https://space-facts.com/mars/"

In [103]:
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [104]:
type(tables)

list

In [105]:
df = tables[0]

df.rows = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons','Orbit Period', 'Surface Temperature', 'First Record', 'Recorded By']
df.columns = ["Property", "Value"]
df.head(9)

/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [106]:
#make dataframes as html
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Property</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC

In [107]:
#strip new lines
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Property</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomer

In [108]:
#save table
df.to_html('table.html')

In [109]:
#open file
!open table.html

# Mars Hemispheres

In [125]:
usgs_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(usgs_url)

html = browser.html
usgs_soup = BeautifulSoup(html, 'html.parser')

mars_hemi = []

In [131]:
# begin rightous for loop 

for i in range(4): 
    time.sleep(5)
    images = browser.find_by_tag("h3")
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    part = soup.find("img", class_="wide-image")["src"]
    title = soup.find("h2", class_ = "title").text
    usg_url = "https://astrogeology.usgs.gov" + part
    dicey ={"title":title, "usg_url":usg_url}
    mars_hemi.append(dicey)
    browser.back()

In [133]:
mars_hemi

[dict,
 dict,
 dict,
 dict,
 {'title': 'Cerberus Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'usg_url': 'https://astrogeology.usgs.gov/cache/images/3